In [ ]:
#!/usr/bin/python3
#from https://stackoverflow.com/a/34132900/3525188
import sys
import fontforge

In [ ]:
import mimetypes
import os
rootdir = '../'
alltextfileconcat=set()
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        #print os.path.join(subdir, file)
        if not any(_ in os.path.join(subdir, file) for _ in ['jmol', '.css']):
            mime = mimetypes.guess_type(file)
            if mime[0] is not None:
                if 'text' in mime[0]:
                    print(os.path.join(subdir, file))
                    f = open(os.path.join(subdir, file), 'r')
                    alltextfileconcat = alltextfileconcat.union(list(f.read()))
                    f.close()

In [ ]:
len(alltextfileconcat)

In [ ]:
for c in [str(_) for i,_ in enumerate(list(alltextfileconcat))]:
    print(c, end=",")


In [ ]:
fontdir = '../_fonts'
targetfontdir = '../_fonts/minified/'

for myfont in os.listdir('../_fonts/'):
    if myfont not in targetfontdir:
        fontpath =  os.path.join(fontdir, myfont)
        targetfontpath = os.path.join(targetfontdir, myfont)
        font = fontforge.open(fontpath)
        for i in alltextfileconcat:
            print(i, ord(i))
            font.selection[ord(i)] = True

        font.selection.invert()

        for i in font.selection.byGlyphs:
            font.removeGlyph(i)
        print("generating font")
        # TODO(firsching): figure out why this hangs!
        font.generate(targetfontpath)

In [ ]:
import base64
ugly={}
types=['R', 'RI', 'RB']
for typ in types:
    f = open(os.path.join(targetfontdir, "LinBiolinum_%s.woff"% typ), 'rb')
    filestr = base64.b64encode((f.read()))   
    f.close()

    # print(filestr.decode())
    print(len(filestr))
    ugly[typ] = filestr.decode()

In [ ]:

biolinum_css = r"""/*
 * CSS-ified version of Biolinum.
 */

@font-face {
    font-family: 'Linux Biolinum'; /* normal */
    src: url('data:application/x-font-woff;base64,%s') format('woff');
    font-weight: normal;
    font-style: normal;
}

@font-face {
    font-family: 'Linux Biolinum'; /* italic */
    src: url('data:application/x-font-woff;base64,%s') format('woff');
    font-weight: normal;
    font-style: italic;
}

@font-face {
    font-family: 'Linux Biolinum'; /* bold */
    src: url('data:application/x-font-woff;base64,%s') format('woff');
    font-weight: bold;
    font-style: normal;
}"""% tuple([ugly[_] for _ in types])

print(biolinum_css)

In [ ]:
biofile = open("../_sass/_biolinum.scss",'w')
biofile.write(biolinum_css)
biofile.close()

In [ ]:
ugly['R']